In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import trange

from skopt import Optimizer
from skopt.plots import plot_convergence, plot_objective, plot_evaluations
from skopt.space import Real

import math

from IPython.display import display
import ipywidgets as widgets

from circle_sphere import circ_to_sphere

## Coordinate System Convention

* +X is right
* +Y is forward
* +Z is up

# Define Double Wishbone Hardpoints


These are the Six 3D points that fully define the shape of the upper and lower wishbone, plus the kingpin axis length.

* A forward and a rear point define the pivot axis of each wishbone.
* The kingpin point finishes the triangle that defines the wishbone.
* The distance between the upper and lower kingpin points fixes the length of the kingpin axis.

Extra Points:
* Wheel center should be on the axis of wheel rotation at the center of the wheel width
* Spindle Point should another ride height point on the wheel spindle axis

> I will likely port my hardpoint optimizer from [c++](https://github.com/cwru-baja/SimulatedAnnealingSuspensionOptimization) to python, which you could use as the first step in a full double wishbone suspension optimization flow.

In [ ]:
column_labels = [widgets.Label(txt) for txt in ('Hardpoint', 'X Coord', 'Y Coord', 'Z Coord')]

row_labels = [widgets.Label(txt) for txt in ('Upper Rear',
                                             'Upper Forward',
                                             'Upper KP',
                                             'Lower Rear',
                                             'Lower Forward',
                                             'Lower KP',
                                             'Wheel Center',
                                             'Spindle Point',
                                             )]

defaults = np.array([
  (0, 0, 6),  # UR 
  (0, 16, 6), # UF 
  (18, 8, 6), # UK 
  (0, 2, 0), # LR 
  (0, 18, 0), # LF 
  (18, 10, 0), # LK 
  (20, 9, 3.0), # Wheel Center
  (18, 9, 3.0), # Spindle Point
])

hardpoint_table = []

table_body = []
for r, row_label in enumerate(row_labels):
  table_body.append(row_label)

  for c in range(3):
    table_cell = widgets.FloatText(value=defaults[r, c])
    table_body.append(table_cell)
    hardpoint_table.append(table_cell)

items = column_labels + table_body
display(widgets.GridBox(items,
                layout=widgets.Layout(grid_template_columns="repeat(4, 150px)")))

## Define Upward and Downward Travel from Ride Height

In [ ]:
display(widgets.FloatText(description='Compression', value=10))
display(widgets.FloatText(description='Full Droop', value=5))

In [ ]:
ride_seven = df.iloc[0:8, 4:7]
shock_travels = df.iloc[4, 1:10]
sp_upright_raw = np.array(df.iloc[17, 4:7], dtype='float64')
sp_upright_rad = 0.5 * 1.20 # Josh approved 20% increase
rack_travel_raw = 2.2253
steering_rack_bounds = df.iloc[9:17, 1:4]

forward = np.array(steering_rack_bounds.iloc[1] - steering_rack_bounds.iloc[0], dtype='float64')
top_forward = np.array(steering_rack_bounds.iloc[3] - steering_rack_bounds.iloc[2], dtype='float64')
up = np.array(steering_rack_bounds.iloc[2] - steering_rack_bounds.iloc[0], dtype='float64')

kp_upper_resting = np.array(ride_seven.iloc[4], dtype='float64')
kp_len = np.linalg.norm(ride_seven.iloc[5] - ride_seven.iloc[4])

In [ ]:
lower_rear = np.array(ride_seven.iloc[2], dtype='float64')
lower_front = np.array(ride_seven.iloc[3], dtype='float64')
kp_lower_resting = np.array(ride_seven.iloc[5], dtype='float64')

wheel = np.array(ride_seven.iloc[6], dtype='float64')
spindle = np.array(ride_seven.iloc[7], dtype='float64')


lower_r_f = lower_front - lower_rear
lower_r_f /= np.linalg.norm(lower_r_f)

# Project rear -> kp onto rear -> front
dot = np.dot(kp_lower_resting - lower_rear, lower_r_f)
circ_center = lower_rear + dot * lower_r_f

circ_radius = np.linalg.norm(kp_lower_resting - circ_center)

circ_center, circ_radius, np.dot(kp_lower_resting - circ_center, lower_r_f)

kp_l_u = kp_upper_resting - kp_lower_resting
kp_l_u /= np.linalg.norm(kp_l_u)

### Write in Rear Wheel Values

## Optimization

In [ ]:
bounds = [
  Real(steering_rack_bounds.min().iloc[0], steering_rack_bounds.max().iloc[0], name="steering rack x"),
  Real(0, 1, name="steering rack u"),
  Real(0, 1, name="steering rack v"),
  Real(0, sp_upright_rad, name="upright sp radius"),
  Real(0, 2 * np.pi, name="upright sp theta"),
  Real(0, np.pi, name="upright sp phi"),
  Real(rack_travel_raw - 0.25, rack_travel_raw + 0.25, name="rack travel"),
]

bounds

In [ ]:
def structify_arg(arg):
  x, u, v, rad, theta, phi, rack_travel = arg

  bb_corner = np.array(steering_rack_bounds.iloc[0], dtype='float64')
  steering_rack_point = np.array([x - bb_corner[0], 0, 0]) + u * (forward * (1 - v) + top_forward * v) + v * up + bb_corner

  outboard_point = sp_upright_raw + rad * np.array([
    np.sin(theta) * np.cos(phi),
    np.sin(theta) * np.sin(phi),
    np.cos(theta),
  ])

  assert np.linalg.norm(outboard_point - sp_upright_raw) <= sp_upright_rad + 1e-10

  return steering_rack_point, outboard_point, rack_travel

# Check the Bounding Box Paramaterization
for corner in range(8):
  bb_corner = np.array(steering_rack_bounds.iloc[corner], dtype='float64')

  match_found = False

  for x in (bounds[0].low, bounds[0].high):
    for u in (0, 1):
      for v in (0, 1):
        synth_point, _, _ = structify_arg((x, u, v, 0, 0, 0, rack_travel_raw))

        current_match = np.allclose(synth_point, bb_corner, atol=1e-8)
        match_found |= current_match
        if current_match:
          print(f"Match @ {x} {u} {v} for {bb_corner} ")
  
  assert match_found, "No match found for {}".format(bb_corner)

## Solve a particular steering geom

### Find the lower kp point at droop and ext

In [ ]:
def solve_given_upper(kp_upper):
  kp_lower, _ = circ_to_sphere(kp_upper, kp_len, circ_center, circ_radius, lower_r_f)
  
  for anchor in (lower_rear, lower_front):
      assert math.isclose(np.linalg.norm(kp_lower - anchor), np.linalg.norm(kp_lower - anchor)), f"|{np.linalg.norm(pos - anchor)}| != |{np.linalg.norm(kp_lower - anchor)}|" 

  return kp_lower

In [ ]:
def evaluate_sp_pair(sp_inboard, sp_upright, rack_travel):
  # print(f"rack travel: {rack_travel}")
  # Project Upright SP -> KP
  sp_kp_comp = np.dot(sp_upright - kp_lower_resting, kp_l_u)
  sp_kp_alpha = sp_kp_comp / np.linalg.norm(kp_upper_resting - kp_lower_resting)

  sp_kp = sp_kp_comp * kp_l_u + kp_lower_resting
  sp_hat_resting = sp_upright - sp_kp
  sp_radius = np.linalg.norm(sp_hat_resting)
  sp_hat_resting /= sp_radius

  # print(f"sp_hat_resting: {sp_hat_resting}")
  
  spin_hat = wheel - spindle
  spin_hat /= np.linalg.norm(spin_hat)

  # Construct a 3D Basis for the spindle axis
  sp_cross_kp = np.cross(sp_hat_resting, kp_l_u)
  sp_cross_kp /= np.linalg.norm(sp_cross_kp)
  # print(f"sp_cross_kp: {sp_cross_kp}")

  basis_matrix = np.vstack((sp_hat_resting, sp_cross_kp, kp_l_u))
  
  spindle_axis_coords = basis_matrix @ spin_hat
  wheel_center_coords = basis_matrix @ (wheel - kp_lower_resting)
  
  # Project the wheel center into this coord system

  toes = np.zeros((3, 3))
  cambers = np.zeros((3, 3))
  wheel_centers = np.zeros((3, 3, 3))
  ackermen = np.zeros((3,))
    
  tie_rod_len = np.linalg.norm(sp_inboard - sp_upright)

  for ind, (tag, rear_pos) in enumerate(zip(('Droop', 'Ride', 'Compress'), rear_wheel_positions)):
      kp_upper = np.array(shock_travels[3 * ind:3 * (ind + 1)], dtype='float64')
      kp_lower = solve_given_upper(kp_upper)

      # print(f"Target {kp_lower} Current {kp_lower}")

      # Compute steering across 3 Wheel Points
      for steer_ind, rack in enumerate(np.linspace(-0.5, 0.5, 3)):
          inboard = sp_inboard + np.array((rack_travel, 0, 0)) * rack

          # print(f"{tag} {rack}")

          # Solve For sp_steered
          sp_anchor = kp_upper * sp_kp_alpha + (1 - sp_kp_alpha) * kp_lower
          
          kp_hat = kp_upper - kp_lower
          kp_hat /= np.linalg.norm(kp_hat)
          
          
          _, sp = circ_to_sphere(inboard, tie_rod_len, sp_anchor, sp_radius, kp_hat)
              
          # Create our coord system
          sp_hat = sp - sp_anchor
          sp_hat /= np.linalg.norm(sp_hat)
          # print(f"sp_hat target: {sp_hat_resting} actual {sp_hat}")
              
          cross_hat = np.cross(sp_hat, kp_hat)
          # print(f"cross_hat target: {sp_cross_kp} actual {cross_hat}")
          
          coord_to_head = np.c_[sp_hat, cross_hat, kp_hat]

          spin_hat = coord_to_head @ spindle_axis_coords
          # print(f"Spin Hat {spin_hat}")
          wheel_centers[ind, steer_ind] = coord_to_head @ wheel_center_coords + kp_lower

          # Camber
          # Positive Camber is the extent to which the spindle axis points in the negative y direction
          # Fully down is 90 degrees, fully up is -90 degrees 
          # Pure steer doesn't change things since that just rotates the spindle about the y axis 
          j_hat = np.array((0, 1, 0)) 
          camber = np.degrees(np.pi / 2
                              - np.arccos(
                                  np.dot(
                                    spin_hat,
                                    -j_hat
                                    )
                                )
                            )
          
          # Steering Angle is the extent to which the wheel plane aligns with the z axis
          # With no camber, the dot product of the z axis and the spindle axis provides a proxy for the steering angle
          # The wheel forward axis should be immune to pure camber effects 
          # So work with the cross product of the up (y) and steering axis
          # 
          # Mind the left handed coord system 
          wheel_forward = np.cross(spin_hat, j_hat)
          wheel_forward /= np.linalg.norm(wheel_forward)
          # print(f"Wheel_forward {wheel_forward}")


          # Define positive steer as outward and negative steer as inward
          i_hat = np.array((1, 0, 0))
          steer_alignment = np.dot(wheel_forward, i_hat)
          # print(f"Steer Alignment {steer_alignment}")
          steer_angle = np.degrees(np.pi / 2 - np.arccos(steer_alignment))
          # print(f"Steer Angle {steer_angle}")


          toes[ind, steer_ind] = steer_angle
          cambers[ind, steer_ind] = camber
          # print()
      
      # Compute Ackermann at this level of compression
      # Compute track_width and wheelbase at center steer 
      RIDE_IND = 1 
      track_width = 2 *wheel_centers[ind, RIDE_IND, 0]
      wheelbase = np.dot(np.array((0, 0, 1)), wheel_centers[ind, RIDE_IND] - rear_pos)
      # print(f"Width {track_width} base {wheelbase}")
      ack_tan = np.arctan2(wheelbase, wheelbase / np.tan(- np.radians(toes[ind, 0])) - track_width)
      # Make it a percent
      directionality = toes[ind, 2] + toes[ind, 0]

      ackermen[ind] = np.radians(toes[ind, 2]) / ack_tan * 100 * abs(directionality) / directionality

      # print()
  
  return toes, ackermen, cambers

np.set_printoptions(precision=3, suppress=True, sign=' ')
sp_inboard_testing = np.array((-0.0, 10.0, 15.0))
toes, ackermen, camber = evaluate_sp_pair(sp_inboard_testing, sp_upright_raw, rack_travel_raw)
toes, ackermen, camber

In [ ]:
def verify_bumps(toes):
  return toes[2] - toes[0]
verify_bumps(toes)

In [ ]:
FAIL_VALUE = 1e4

def obj_func(x, exp=2):
  inboard, outboard, rack_travel = structify_arg(x)

  try:
    toes, ackermen, cambers = evaluate_sp_pair(inboard, outboard, rack_travel)
  except Exception as e:
    return FAIL_VALUE
  
  droups = toes[0] 
  # rides = toes[1] 
  comps = toes[2]
  bumps = comps - droups

  # Ignore droop Ackermann
  weighted_error = np.hstack([bumps,
                              0.125 * (ackermen - ackermann_targets)[1],
                              0.25 * (ackermen - ackermann_targets)[2],
                              2 * (toes[0, 2] - full_droop_inside)])
  return np.sum(np.abs(weighted_error) ** exp)

# Running SkOpt

In [27]:
def argmin(a):
    return min(range(len(a)), key=lambda x : a[x])

def argmax(a):
    return max(range(len(a)), key=lambda x : a[x])

def validate_marg(optimizer):
  marg = optimizer.Xi[argmin(optimizer.yi)]
  struct = structify_arg(marg)
  toes, ackermen, cambers = evaluate_sp_pair(*struct)

  print(f"marg:\n{marg}")
  print(f"inboard point:\n{struct[0]}")
  print(f"outboard point:\n{struct[1]}")
  print(f"toes:\n{toes}")
  print(f"Ackermen:\n{ackermen}")
  print(f"cambers:\n{cambers}")
  print(f"bumps:\n{verify_bumps(toes)}")
  print()

In [ ]:
for exp in (2,): 
  optimizer = Optimizer(
      dimensions=bounds,
      # random_state=1,
      base_estimator='gp',
      # Cost to evaluate func
      #  et rf ?
      #  <= GBRT <= GP
  )
  for iter in trange(1_500):
    arg = optimizer.ask()
    result = obj_func(arg, exp=exp)
    optimizer.tell(arg, result)
  
  
  objective = plot_objective(optimizer.get_result(), n_points=10, sample_source='result', show_points=False) 
  plt.show()
  print(f"Obj L{exp} norm")
  validate_marg(optimizer)

In [26]:
convergence = plot_convergence(optimizer.get_result(), yscale='log')

NameError: name 'optimizer' is not defined

In [ ]:
evaluations = plot_evaluations(optimizer.get_result())

In [25]:
objective = plot_objective(optimizer.get_result(),
                          #  n_points=10,
                           levels=20,
                           sample_source='result',
                           show_points=False) 

NameError: name 'optimizer' is not defined